In [8]:

####################
#Author: brandon chiazza
#version: 1.0
#purpose: to call a twitter api and return results
#documentation: https://developer.twitter.com/en/docs
#####################

import pandas as pd
import requests
import json
import base64
!pip install s3fs
import s3fs # documentation: https://s3fs.readthedocs.io/en/latest/
import time
import twitter_keys #this is a custom reference module to a package containing twitter keys

%config IPCompleter.greedy=True


key_secret = '{}:{}'.format(twitter_keys.client_key, twitter_keys.client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

#identify base url and oauth token path
base_url = 'https://api.twitter.com/' #base url for authentication
auth_url = '{}oauth2/token'.format(base_url)

#share header information -- encoding is ascii
auth_headers = {
    'Authorization': 'Basic {}'.format(b64_encoded_key),
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
}

#pass clientcredentials
auth_data = {
    'grant_type': 'client_credentials'
}

#send authentication using requests - POST request
auth_resp = requests.post(auth_url, headers=auth_headers, data=auth_data)

#check response status. 200 = OK
auth_resp.status_code




  Created wheel for aiobotocore: filename=aiobotocore-2.2.0-py3-none-any.whl size=57108 sha256=cb0dcc9a6cb55369fdcba4debeb95acdc8934fa9b8854150799949b2c3681364
  Stored in directory: c:\users\brand\appdata\local\pip\cache\wheels\03\00\b5\78eda0a8dffd05e210f46e99fd8df6775f76cca046f7a3a5fc
Successfully built aiobotocore
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.10.1
    Uninstalling fsspec-2021.10.1:
      Successfully uninstalled fsspec-2021.10.1


200

In [9]:

#Keys in data response are token_type (bearer) and access_token (your access token)
print(auth_resp.json().keys())

access_token = auth_resp.json()['access_token']


search_headers = {
    'Authorization': 'Bearer {}'.format(access_token)    
}

#enter search parameters for coronavirus example. This looks for "covid-19" in the 1000 most recent tweets
query_params = {
    'q': 'covid-19',
    'result_type': 'recent',
    'count': 100, #update here to get more/less than 1000 returns
    'lang': 'en' #filters by english language only
}


#identify search url path and save 
search_url = '{}1.1/search/tweets.json'.format(base_url)


#run search using get request
search_resp = requests.get(search_url, headers=search_headers, params=query_params)

#check status code of GET request
search_resp.status_code


dict_keys(['token_type', 'access_token'])


200

In [10]:
#print text from result to verify  
twitter_data = search_resp.json()

for x in twitter_data['statuses']:
    print(x['text'] + '\n')
    break #prints after one iteration and stops, remove break to see all 1000

Experts say it's too early to anticipate a sixth wave of COVID-19 in B.C. Layers of protection like vaccines and ma… https://t.co/UKn8WgMxjZ



In [11]:
# move data into data frame 
df = pd.DataFrame(twitter_data['statuses'])

# show one record to verify import 
df.head(1)

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,retweeted_status,extended_entities,quoted_status_id,quoted_status_id_str
0,Thu Mar 31 23:06:56 +0000 2022,1509668598364606478,1509668598364606478,Experts say it's too early to anticipate a six...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,0,0,False,False,False,en,NaN,NaN,NaN,NaN


In [13]:
# we can use pandas to put data directly into an s3 bucket
#prepare csv file name   
filename = 's3/lab-03/'#specify location of s3:/{my-bucket}/
groupname= 'Group_test_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(filename,groupname,datetime) #name of the filepath and csv file



#encoding must be adjusted to accommodate abnormal characters. Use s3fs to write to S3 bucket
byte_encoded_df = df.to_csv(None).encode() #encodes file as binary
s3 = s3fs.S3FileSystem(anon=False)
with s3.open(filenames3, 'wb') as file:
    file.write(byte_encoded_df) #writes byte-encoded file to s3 location

#print success message
print("Successfull uploaded file to location:"+str(filenames3))


NoCredentialsError: Unable to locate credentials